https://myoptimizationnotes.blogspot.com/2024/01/data-envelopment-analysis.html 

# Using Specific Example

In [1]:
from pulp import *

In [8]:
import pulp

# Create a linear programming problem
prob = pulp.LpProblem("DEA", pulp.LpMaximize)

# Define variables for lambda (λ) and mu (μ)
lambda_var = pulp.LpVariable("Lambda", lowBound=0)
mu_var = pulp.LpVariable("Mu", lowBound=0)

# Define the original data for each plant
original_data = {
    'Plant A': {'E1': 10000, 'E2': 200, 'O1': 1000, 'O2': 20000},
    'Plant B': {'E1': 8000, 'E2': 250, 'O1': 800, 'O2': 16000},
    'Plant C': {'E1': 12000, 'E2': 300, 'O1': 1200, 'O2': 24000}
}

# Normalize the data
normalized_data = {}
for plant, data in original_data.items():
    max_E1 = max([data['E1'] for data in original_data.values()])
    max_E2 = max([data['E2'] for data in original_data.values()])
    max_O1 = max([data['O1'] for data in original_data.values()])
    max_O2 = max([data['O2'] for data in original_data.values()])
    
    normalized_data[plant] = {
        'E1': data['E1'] / max_E1,
        'E2': data['E2'] / max_E2,
        'O1': data['O1'] / max_O1,
        'O2': data['O2'] / max_O2}
    print(plant,':E1',normalized_data[plant]['E1'],'E2',normalized_data[plant]['E2'],'O1',normalized_data[plant]['O1'],
     'O2',normalized_data[plant]['O2'])

# Define the objective function to maximize ES while minimizing mu
prob += lambda_var + mu_var, "Efficiency Score"

# Define the constraints for all plants
for plant in normalized_data:
    prob += lambda_var * normalized_data[plant]['E1'] + mu_var * normalized_data[plant]['E2'] <= 1, f"Constraint_{plant}"

# Solve the linear programming problem
prob.solve()

# Calculate the efficiency scores for all plants
efficiency_scores = {}
for plant in normalized_data:
    efficiency_scores[plant] = pulp.value(lambda_var) * normalized_data[plant]['O1'] + pulp.value(mu_var) * normalized_data[plant]['O2']

# Print the efficiency scores for all plants
for plant, efficiency_score in efficiency_scores.items():
    print(f"Efficiency Score for {plant}: {efficiency_score:.2f}")

# Print the optimal lambda (λ) and mu (μ)
print(f"Optimal Lambda (λ): {pulp.value(lambda_var)}")
print(f"Optimal Mu (μ): {pulp.value(mu_var)}")


Plant A :E1 0.8333333333333334 E2 0.6666666666666666 O1 0.8333333333333334 O2 0.8333333333333334
Plant B :E1 0.6666666666666666 E2 0.8333333333333334 O1 0.6666666666666666 O2 0.6666666666666666
Plant C :E1 1.0 E2 1.0 O1 1.0 O2 1.0
Efficiency Score for Plant A: 0.83
Efficiency Score for Plant B: 0.67
Efficiency Score for Plant C: 1.00
Optimal Lambda (λ): 0.0
Optimal Mu (μ): 1.0


# Using General Examples

In [9]:
import pulp

def perform_dea_analysis(data):
    # Create a linear programming problem
    prob = pulp.LpProblem("DEA", pulp.LpMaximize)

    num_inputs = len(data[0]['inputs'])
    num_outputs = len(data[0]['outputs'])

    # Define variables for lambda (λ) and mu (μ)
    lambda_var = pulp.LpVariable("Lambda", lowBound=0)
    mu_var = pulp.LpVariable("Mu", lowBound=0)

    # Normalize the data
    normalized_data = normalize_data(data)

    # Define the objective function to maximize ES while minimizing mu
    prob += lambda_var - mu_var, "Efficiency Score"

    # Define the constraints for all plants
    for plant_data in normalized_data:
        inputs = plant_data['inputs']
        outputs = plant_data['outputs']

        input_constraints = [lambda_var * inputs[i] for i in range(num_inputs)]
        output_constraints = [mu_var * outputs[i] for i in range(num_outputs)]

        prob += sum(input_constraints) + sum(output_constraints) <= 1, f"Constraint_{plant_data['name']}"

    # Solve the linear programming problem
    prob.solve()

    # Calculate the efficiency scores for all plants
    efficiency_scores = {}
    for plant_data in normalized_data:
        inputs = plant_data['inputs']
        outputs = plant_data['outputs']

        efficiency_score = sum([pulp.value(lambda_var) * inputs[i] for i in range(num_inputs)]) \
            + sum([pulp.value(mu_var) * outputs[i] for i in range(num_outputs)])

        efficiency_scores[plant_data['name']] = efficiency_score

    # Print the efficiency scores for all plants
    for plant, efficiency_score in efficiency_scores.items():
        print(f"Efficiency Score for {plant}: {efficiency_score:.2f}")

    # Print the optimal lambda (λ) and mu (μ)
    print(f"Optimal Lambda (λ): {pulp.value(lambda_var)}")
    print(f"Optimal Mu (μ): {pulp.value(mu_var)}")

def normalize_data(data):
    max_inputs = [max(plant['inputs']) for plant in data]
    max_outputs = [max(plant['outputs']) for plant in data]

    normalized_data = []
    for plant in data:
        inputs = [x / max_x for x, max_x in zip(plant['inputs'], max_inputs)]
        outputs = [x / max_x for x, max_x in zip(plant['outputs'], max_outputs)]

        normalized_data.append({
            'name': plant['name'],
            'inputs': inputs,
            'outputs': outputs
        })

    return normalized_data

# Define the data for each plant (you can add more plants as needed)
plants_data = [
    {
        'name': 'Plant A',
        'inputs': [10000, 200],
        'outputs': [1000, 20000]
    },
    {
        'name': 'Plant B',
        'inputs': [8000, 250],
        'outputs': [800, 16000]
    },
    {
        'name': 'Plant C',
        'inputs': [12000, 300],
        'outputs': [1200, 24000]
    }
]

# Perform DEA analysis
perform_dea_analysis(plants_data)


Efficiency Score for Plant A: 0.83
Efficiency Score for Plant B: 0.67
Efficiency Score for Plant C: 1.00
Optimal Lambda (λ): 0.80808081
Optimal Mu (μ): 0.0
